In [ ]:
import arcpy
import pandas as pd

PROJ_PATH = "C:/Users/bento/final_project/woodpeckerNC/woodpeckerNC.aprx"
proj = arcpy.mp.ArcGISProject(PROJ_PATH)
DB_PATH = proj.defaultGeodatabase 
arcpy.env.workspace = DB_PATH

sensitivity_table = "Pileated_Woodpecker_NC_Sensitivity_Table_1"

field_names = [field.name for field in arcpy.ListFields(sensitivity_table)]
with arcpy.da.SearchCursor(sensitivity_table, 
                           ['CUTOFF', 'FPR', 'TPR', 'FNR', 'TNR', 'SENSE', 'SPEC']) as cursor:
    vals = [
        {
            "cutoff": cutoff,
            "FP": FP,
            "TP": TP,
            "FN": FN,
            "TN": TN,
            "recall": recall,
            "specificity": specificity,
            "precision": TP / (TP + FP),
            "f1": 2 * ((TP / (TP + FP)) * recall) / ((TP / (TP + FP)) + recall),
        }
        for (cutoff, FP, TP, FN, TN, recall, specificity) in cursor
    ]
df = pd.DataFrame.from_dict(vals)

print(f"Max F1: {round(max(df.f1), 4)}")
print(f"Best Cutoff: {round(df.loc[df.f1 == max(df.f1)].cutoff.iloc[0], 2)}")

df


In [6]:
import os
import pickle

model_data_path = "C:/Users/bento/final_project/woodpeckerNC/data/model_data"
s = "Pileated_Woodpecker"
file = os.path.join(model_data_path, f"{s}_model_data.pickle")
print(file)

with open(file, "rb") as f:
    cached_model_data = pickle.load(f)

cached_model_data['params']

C:/Users/bento/final_project/woodpeckerNC/data/model_data\Pileated_Woodpecker_model_data.pickle


{'number_of_iterations': 20,
 'basis_expansion_functions': 'THRESHOLD',
 'relative_weight': 50,
 'number_knots': 50,
 'spatial_thinning': 'THINNING',
 'link_function': 'CLOGLOG',
 'thinning_distance_band': '2500 Meters'}

In [ ]:
log_path = os.path.join(model_data_path, "model_training_logs")

all_params = 
files = os.listdir(log_path)
    for file in files:
        file_path = os.path.join(log_path, file)
        with open(file_path, 'r') as f:
            file_dict = json.load(f)

In [12]:
import itertools

parameter_grid = {
    "number_of_iterations": [10, 20],
    "basis_expansion_functions":["HINGE", "LINEAR", "THRESHOLD", "QUADRATIC"], # "PRODUCT"
    "relative_weight": [35, 50, 100],
    "number_knots":[10, 50], 
    "spatial_thinning": ["NO_THINNING", "THINNING"],
    "link_function": ["CLOGLOG"], # "LOGISTIC"
    "thinning_distance_band": ["1000 Meters", "2500 Meters"] # "5000 Meters"
}
def getAllCombos(parameter_grid:dict) -> dict:
    # Generate all combinations of parameters
    if "NO_THINNING" not in parameter_grid["spatial_thinning"]:
        all_combinations = list(itertools.product(*parameter_grid.values()))
    elif "THINNING" not in parameter_grid["spatial_thinning"]:
        # Generate combinations for NO_THINNING
        all_combinations = [
            (None, basis_expansion, weight, knots, "NO_THINNING", link_function, None)
            for basis_expansion, weight, knots, link_function
            in itertools.product(
                parameter_grid["basis_expansion_functions"],
                parameter_grid["relative_weight"],
                parameter_grid["number_knots"],
                parameter_grid["link_function"]
            )
        ]
    else:
        # Generate combinations for NO_THINNING
        no_thinning_combinations = [
            (None, basis_expansion, weight, knots, "NO_THINNING", link_function, None)
            for basis_expansion, weight, knots, link_function
            in itertools.product(
                parameter_grid["basis_expansion_functions"],
                parameter_grid["relative_weight"],
                parameter_grid["number_knots"],
                parameter_grid["link_function"]
            )
        ]
        # Generate combinations for THINNING
        thinning_combinations = list(itertools.product(
            parameter_grid["number_of_iterations"],
            parameter_grid["basis_expansion_functions"],
            parameter_grid["relative_weight"],
            parameter_grid["number_knots"],
            ["THINNING"],
            parameter_grid["link_function"],
            parameter_grid["thinning_distance_band"]
        ))
        # Combine the two lists of combinations
        all_combinations = no_thinning_combinations + thinning_combinations
    return all_combinations

len(getAllCombos(parameter_grid))

120

In [2]:

import os
import arcpy 

data_path="C:/Users/bento/final_project/woodpeckerNC/data"
wspace="C:/Users/bento/final_project/woodpeckerNC/woodpeckerNC.gdb"
coord_sys = arcpy.SpatialReference("NAD 1983 StatePlane North Carolina FIPS 3200 (US Feet)")

# Set workspace
arcpy.env.workspace = wspace

# Get NC Boundary Shape File if not downloaded
nc_shp_path = os.path.join(data_path, "nc_state_boundary")
nc_shp_file = os.path.join(nc_shp_path, "tl_2018_37_cousub.shp")
os.makedirs(nc_shp_path, exist_ok=True)

In [4]:
arcpy.management.Dissolve(nc_shp_file , "nc_state_boundary", None, None, 
                                "SINGLE_PART", "DISSOLVE_LINES")

<Result 'C:/Users/bento/final_project/woodpeckerNC/woodpeckerNC.gdb\\nc_state_boundary'>

In [5]:
coord_sys

name (Projected Coordinate System),NAD_1983_StatePlane_North_Carolina_FIPS_3200_Feet
factoryCode (WKID),2264
linearUnitName (Linear Unit),Foot_US
name (Geographic Coordinate System),GCS_North_American_1983
factoryCode (WKID),4269
angularUnitName (Angular Unit),Degree
datumName (Datum),D_North_American_1983


In [1]:
# Add to GDB if needed
arcpy.management.DefineProjection("nc_state_boundary_projected", coord_sys)


NameError: name 'brd_rasters' is not defined

In [6]:
arcpy.SpatialReference().loadFromString('PROJCS["NAD_1983_StatePlane_North_Carolina_FIPS_3200_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",2000000.002616666],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-79.0],PARAMETER["Standard_Parallel_1",34.33333333333334],PARAMETER["Standard_Parallel_2",36.16666666666666],PARAMETER["Latitude_Of_Origin",33.75],UNIT["Foot_US",0.3048006096012192]]') == coord_sys

False

In [11]:
coord_sys.exportToString()

'PROJCS["NAD_1983_StatePlane_North_Carolina_FIPS_3200_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",2000000.002616666],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-79.0],PARAMETER["Standard_Parallel_1",34.33333333333334],PARAMETER["Standard_Parallel_2",36.16666666666666],PARAMETER["Latitude_Of_Origin",33.75],UNIT["Foot_US",0.3048006096012192]];-121841900 -93659000 3048.00609601219;-100000 10000;-100000 10000;3.28083333333333E-03;0.001;0.001;IsHighPrecision'

In [8]:
coord_sys.exportToString() == 'PROJCS["NAD_1983_StatePlane_North_Carolina_FIPS_3200_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",2000000.002616666],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-79.0],PARAMETER["Standard_Parallel_1",34.33333333333334],PARAMETER["Standard_Parallel_2",36.16666666666666],PARAMETER["Latitude_Of_Origin",33.75],UNIT["Foot_US",0.3048006096012192]]'


False